In [12]:
import sys
import os
sys.path.append('src')

In [14]:
from sklearn.datasets import fetch_20newsgroups
from src.cleaning.cleaners import BasicTextCleaner
from tokenisation.tokenisers import BasicTokeniser
from word_encoding.word_encoders import BasicEncoder

from itertools import chain

ModuleNotFoundError: No module named 'src'

In [5]:
import os

os.getcwd()

'/workspace/src/notebooks'

In [ ]:
data = fetch_20newsgroups(subset='train')

X = data.data
y = data.target

In [ ]:
# Optional: Get the category names
label_names = data.target_names

cleaner = BasicTextCleaner()
X_clean = [cleaner.clean_text(text) for text in X]

print(X_clean)

In [ ]:
tokeniser = BasicTokeniser()
X_tokens = [tokeniser.tokenise(text) for text in X_clean]

print(X_tokens)

In [ ]:
max_doc_len = max(X_tokens, key=len)

flat = list(chain.from_iterable(X_tokens))
encoder = BasicEncoder(flat)
X_encodings = [encoder.encode(doc, max_len=max_doc_len) for doc in X_tokens]
print(X_encodings)